In [ ]:
__author__ = 'Benjamin Weaver <benjamin.weaver@noirlab.edu>'
__version__ = '202100302'
__datasets__ = ['sdss_dr16']
__keywords__ = ['HowTo','spectra','query','visualization','SDSS']

# How to Plot Spectral Data with Prospect, Specutils and the Astro Data Lab Spectrum Service

## Table of Contents

* [Goals](#Goals)
* [Summary](#Summary)
* [Disclaimer & Attribution](#Disclaimer-&-Attribution)
* [Imports & Setup](#Imports-&-Setup)
* [Authentication](#Authentication)
* [SDSS/eBOSS spPlate file](#SDSS/eBOSS-spPlate-file)
  - [Find Plate 2955 in the Spectrum Service](#Find-Plate-2955-in-the-Spectrum-Service)
  - [Spectrum Identifiers](#Spectrum-Identifiers)
  - [Spectrum Service: Flux versus Wavelength](#Spectrum-Service:-Flux-versus-Wavelength)
  - [Redshift and other Metadata](#Redshift-and-other-Metadata)
    * [Redshift Catalog](#Redshift-Catalog)
    * [Target Catalog](#Target-Catalog)
* [Launch the Visualization](#Launch-the-Visualization)
  - [Interface Visualization Details](#Interface-Visualization-Details)
* [SDSS Spectra and Redshifts from Files](#SDSS-Spectra-and-Redshifts-from-Files)
* [References & Resources](#References-&-Resources)

## Goals

Obtain SDSS spectra using the Astro Data Lab spectrum service, convert data to [specutils](https://github.com/astropy/specutils) objects as needed, and use [Prospect](https://github.com/desihub/prospect) to display the data.

## Summary

Prospect is an *interactive* spectrum visualization service that is being actively used by the [DESI](https://www.desi.lbl.gov/) project for visual inspection of commissioning and survey validation spectra right now.  Rather than just a simple *flux* versus *wavelength* plot, the interactive spectrum display provides pan, zoom, known spectral lines, targeting and other catalog information, metadata, *etc.*  The [Bokeh](https://bokeh.org) library handles the low-level interactivity.

We can leverage the generic spectrum container objects provided by [specutils](https://github.com/astropy/specutils) to allow Prospect to plot data from other surveys.  In this example, we are using [SDSS DR16](https://www.sdss.org/dr16/).

In the future we will extend this functionality to additional instruments.  For example, we are excited by the possibility of using Prospect and the Astro Data Lab spectrum service for Gemini spectra.  And, of course, once DESI data become public, both the spectrum service and this tool will be ready.

## Disclaimer & Attribution

If you use this notebook for your published science, please acknowledge the following:

* Astro Data Lab concept paper: Fitzpatrick et al., "The NOAO Data Laboratory: a conceptual overview", SPIE, 9149, 2014, http://dx.doi.org/10.1117/12.2057445
* Astro Data Lab disclaimer: https://datalab.noirlab.edu/disclaimers.php

## Imports & Setup

This cell performs all necessary imports for this notebook.

In [1]:
import os
import sys
import time
from getpass import getpass
#
# Numpy, astropy, etc.
#
import numpy as np
import astropy.units as u
from astropy.table import Table
from astropy.nddata import InverseVariance
from specutils import Spectrum1D, SpectrumCollection, SpectrumList
#
# Prospect imports
#
from prospect.specutils import read_spPlate, read_spZbest
from prospect.viewer import plotspectra
#
# Astro Data Lab imports
#
from dl import specClient as spec  # primary spectral data client interface
from dl import storeClient as sc   # needed to use virtual storage
from dl import queryClient as qc   # needed to query Astro Data Lab catalogs
from dl import authClient as ac    # needed for login authentication

start_time = time.time()  # save start time of notebook run

## Authentication

Much of the functionality of spectrum services can be accessed without explicitly logging into Astro Data Lab (the service then uses an anonymous login). But some capabilities, for instance saving the results of your queries to your virtual storage space, require a login (*i.e.* you will need a [registered user account](https://datalab.noao.edu/help/register.php?qa=register)).

If you need to log in to Astro Data Lab, uncomment the `ac.login()` command and respond according to the prompts.  If you have previously logged into Astro Data Lab, this cell will simply print your active user name.

In [ ]:
# ac.login(input("Enter user name: (+ENTER) "),getpass("Enter password: (+ENTER) "))
ac.whoAmI()

## SDSS/eBOSS spPlate file

SDSS spectra are stored per-plate in spPlate files.  These contain 640 spectra for the original SDSS spectrograph or 1000 spectra for the BOSS/eBOSS spectrograph.  All spectra in an individual spPlate file have a common wavelength solution, so a single spPlate file can be represented by a [Spectrum1D](https://specutils.readthedocs.io/en/stable/api/specutils.Spectrum1D.html#specutils.Spectrum1D) object.  The Prospect visualization package can then take this Spectrum1D object as input data.

### Find Plate 2955 in the Spectrum Service

In the cells below, we will find spectra corresponding to SDSS plate 2955, but using the spectrum service instead of reading the spectra from a file.  This starts by finding the coordinates of the plate center.

In [2]:
plate = 2955
q = f"SELECT ra, dec FROM sdss_dr16.platex WHERE plate = {plate}"
r = qc.query(sql=q, fmt='array')
r

array([235.4204  ,   1.644156])

### Spectrum Identifiers

SDSS plates are circular, so we'll do a cone search around the plate center found above.  This gets us the spectrum identifiers we need to retrieve the spectra themselves.

In [3]:
sdss_ids = spec.query(r[0], r[1], 1.5, fmt='array', out='', constraint=f'plate = {plate} ORDER BY specobjid LIMIT 50')
sdss_ids

array([3327035125891360768, 3327036500280895488, 3327037050036709376,
       3327038424426244096, 3327038974182057984, 3327040073693685760,
       3327040348571592704, 3327040623449499648, 3327040898327406592,
       3327041173205313536, 3327041722961127424, 3327041997839034368,
       3327042822472755200, 3327043097350662144, 3327043372228569088,
       3327043921984382976, 3327044196862289920, 3327045021496010752,
       3327045296373917696, 3327046121007638528, 3327046670763452416,
       3327047495397173248, 3327048045152987136, 3327048320030894080,
       3327048594908801024, 3327048869786707968, 3327049144664614912,
       3327049419542521856, 3327049694420428800, 3327050244176242688,
       3327051068809963520, 3327051343687870464, 3327052443199498240,
       3327052992955312128, 3327053267833219072, 3327053817589032960,
       3327054092466939904, 3327054642222753792, 3327054917100660736,
       3327055191978567680, 3327055466856474624, 3327056016612288512,
       3327056566368

### Spectrum Service: Flux versus Wavelength

Now we fetch the spectra.  The option `align=True` directes the service to return all spectra as a single `Spectrum1D` object with a common wavelength axis for all spectra.  Since all spectra on a single SDSS plate have a common wavelength axis anyway, this is a simple operation.

In [4]:
%%time
sdss = spec.getSpec(sdss_ids, fmt='Spectrum1D', align=True)
sdss

CPU times: user 83.1 ms, sys: 48.7 ms, total: 132 ms
Wall time: 588 ms


<Spectrum1D(flux=<Quantity [[0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.],
           ...,
           [0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.]] erg / (Angstrom cm2 s)>, spectral_axis=<SpectralAxis [3802.77  , 3803.6448, 3804.522 , ..., 9217.223 , 9219.344 , 9221.464 ] Angstrom>, uncertainty=InverseVariance([[0., 0., 0., ..., 0., 0., 0.],
                 [0., 0., 0., ..., 0., 0., 0.],
                 [0., 0., 0., ..., 0., 0., 0.],
                 ...,
                 [0., 0., 0., ..., 0., 0., 0.],
                 [0., 0., 0., ..., 0., 0., 0.],
                 [0., 0., 0., ..., 0., 0., 0.]]))>

### Redshift and other Metadata

The Prospect viewer requires certain spectral metadata that are normally provided with SDSS spectra--*i.e.*, these metadata are stored alongside the flux *versus* wavelength data.  There are two categories of metadata:

1. Redshift catalog: this contains the redshift (how far away it is) and classification (what type of object it is) for each spectrum, as identified by the SDSS pipeline.
2. Target catalog: this contains the information needed for taking a spectrum of the object in the first place.  This includes information about the object as identified in SDSS *imaging* (RA, Dec, optical magnitude), as well as the rationale for taking a spectrum of the object.

In the cells below, we will extract the necessary metadata from the `specobj` and `photoplate` tables and reformat them slightly to match the appearance of these data as they would be extracted from SDSS files.

#### Redshift Catalog

This query extracts the redshift and classification information from the `specobj` table.

In [5]:
q = "SELECT specobjid, class, subclass, z, zerr, rchi2diff, zwarning FROM sdss_dr16.specobj WHERE specobjid IN ({0}) ORDER BY specobjid".format(','.join(map(str, sdss_ids.tolist())))
sdss_z = qc.query(sql=q, fmt='table')
for c in ('specobjid', 'class', 'subclass', 'z', 'zerr', 'rchi2diff', 'zwarning'):
    if c == 'zerr':
        sdss_z.rename_column(c, 'Z_ERR')
    else:
        sdss_z.rename_column(c, c.upper())
sdss_z

SPECOBJID,CLASS,SUBCLASS,Z,Z_ERR,RCHI2DIFF,ZWARNING
int64,str6,str11,float64,float64,float64,int64
3327035125891360768,GALAXY,--,0.262199,6.42647e-05,0.161533,0
3327036500280895488,GALAXY,--,0.21097,5.79199e-05,0.139213,0
3327037050036709376,STAR,K5,-2.34944e-06,1.43502e-05,0.496992,0
3327038424426244096,GALAXY,--,0.260543,6.22988e-05,0.225588,0
3327038974182057984,GALAXY,--,0.112768,1.59947e-05,0.510905,0
3327040073693685760,GALAXY,--,0.0636493,2.83125e-05,0.424366,0
3327040348571592704,GALAXY,--,0.208361,4.18928e-05,1.17941,0
3327040623449499648,GALAXY,--,0.0,0.0,0.0,134
3327040898327406592,GALAXY,--,0.207845,4.12366e-05,0.619729,0


#### Target Catalog

This query extracts photometric targeting information from the `photoplate` table.  The data obtained needs to be reformatted to match the ["plugmap" metadata](https://data.sdss.org/datamodel/files/BOSS_SPECTRO_REDUX/RUN2D/PLATE4/spPlate.html#hdu5head) that is stored with SDSS spectrum files, including spPlate files.

In [6]:
q = """SELECT s.specobjid, s.targetobjid, s.ra, s.dec,
    s.primtarget, s.sectarget,
    s.boss_target1,
    s.ancillary_target1, s.ancillary_target2,
    s.eboss_target0, s.eboss_target1, s.eboss_target2,
    p.u, p.g, p.r, p.i, p.z
FROM sdss_dr16.specobj AS s
LEFT JOIN sdss_dr16.photoplate AS p
ON s.bestobjid = p.objid
WHERE specobjid IN ({0})
ORDER BY specobjid""".format(','.join(map(str, sdss_ids.tolist())))
sdss_plugmap = qc.query(sql=q, fmt='table')
for c in ('ra', 'dec', 'primtarget', 'sectarget',
          'boss_target1',
          'ancillary_target1', 'ancillary_target2',
          'eboss_target0', 'eboss_target1', 'eboss_target2'):
    sdss_plugmap.rename_column(c, c.upper())
sdss_plugmap['OBJID'] = np.vstack((np.bitwise_and(sdss_plugmap['targetobjid'] >> 32, 2**16 - 1).data,
                                   np.bitwise_and(sdss_plugmap['targetobjid'] >> 48, 2**11 - 1).data,
                                   np.bitwise_and(sdss_plugmap['targetobjid'] >> 29, 2**3 - 1).data,
                                   np.bitwise_and(sdss_plugmap['targetobjid'] >> 16, 2**12 - 1).data,
                                   np.bitwise_and(sdss_plugmap['targetobjid'], 2**16 - 1).data)).T
sdss_plugmap['MAG'] = np.vstack((np.where(np.isnan(sdss_plugmap['u'].data), 0, sdss_plugmap['u'].data),
                                 np.where(np.isnan(sdss_plugmap['g'].data), 0, sdss_plugmap['g'].data),
                                 np.where(np.isnan(sdss_plugmap['r'].data), 0, sdss_plugmap['r'].data),
                                 np.where(np.isnan(sdss_plugmap['i'].data), 0, sdss_plugmap['i'].data),
                                 np.where(np.isnan(sdss_plugmap['z'].data), 0, sdss_plugmap['z'].data))).T

sdss.meta['plugmap'] = sdss_plugmap
sdss_plugmap

specobjid,targetobjid,RA,DEC,PRIMTARGET,SECTARGET,BOSS_TARGET1,ANCILLARY_TARGET1,ANCILLARY_TARGET2,EBOSS_TARGET0,EBOSS_TARGET1,EBOSS_TARGET2,u,g,r,i,z,OBJID [5],MAG [5]
int64,int64,float64,float64,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,float64,float64,float64,int64,float64
3327035125891360768,11268994537292320,236.58816,0.91476191,96,0,0,0,0,0,0,0,21.3034,19.1014,17.512,16.9372,16.5634,2327 .. 544,21.3034 .. 16.5634
3327036500280895488,11268994537161253,236.31498,0.83142165,64,0,0,0,0,0,0,0,20.5745,19.0677,17.9301,17.3995,16.9857,2327 .. 549,20.5745 .. 16.9857
3327037050036709376,11268994537095484,236.17657,0.91950364,64,0,0,0,0,0,0,0,19.8611,18.5047,17.7639,17.4456,17.246,2327 .. 316,19.8611 .. 17.246
3327038424426244096,11268994537292314,236.57992,0.84114607,32,0,0,0,0,0,0,0,21.9935,19.606,18.0296,17.4561,17.0533,2327 .. 538,21.9935 .. 17.0533
3327038974182057984,11268994537292219,236.63067,0.85355411,64,0,0,0,0,0,0,0,19.6481,18.0843,17.2542,16.8369,16.5255,2327 .. 443,19.6481 .. 16.5255
3327040073693685760,11265262790246829,236.57247,1.3802039,64,0,0,0,0,0,0,0,20.0069,18.0379,17.1379,16.7259,16.3942,1458 .. 429,20.0069 .. 16.3942
3327040348571592704,11265262790312240,236.66305,1.4061234,96,0,0,0,0,0,0,0,20.1158,17.9848,16.4976,15.9314,15.5032,1458 .. 304,20.1158 .. 15.5032
3327040623449499648,11265262252261744,234.00554,1.1758617,64,0,0,0,0,0,0,0,--,--,--,--,--,1458 .. 368,0.0 .. 0.0
3327040898327406592,11268995074162967,236.57875,1.3024246,64,0,0,0,0,0,0,0,21.2316,18.8937,17.5176,16.9796,16.5934,2327 .. 279,21.2316 .. 16.5934


## Launch the Visualization

Once all the spectral data are assembled, the visualization is launched with a single `plotspectra()` function in the cell below.  The primary input is the Spectrum1D object created above.  Prospect expects a certain scaling on the input flux, so `sdss.new_flux_unit()` handles that (this changes the numerical scale factor on the flux unit, not the flux unit itself).  Other inputs:

* `zcatalog`: the [redshift table](#Redshift-Catalog) created above.
* `model`: the model spectrum used by the SDSS pipeline to obtain the redshift and classification.
* `notebook`: instructs Prospect to use notebook display mode (as opposed to standalone HTML).
* `title`, `model_from_zcat`, etc.: these options control certain display options that are not directly related to the input data or that need to be switched off for SDSS data.  A [full description is available](https://desi-prospect.readthedocs.io/en/stable/api.html#prospect.viewer.plotspectra).

Once the cell is run, it will take several seconds for the data to load and render.  Then you will be able to pan and zoom on the primary flux *versus* wavelength display, page through different spectra, show known emission or absorption lines, etc.  [More details are below the display.](#Visualization-Interface-Details)

In [ ]:
plotspectra(sdss.new_flux_unit(u.Unit('1e-17 erg/(cm**2 s Angstrom)')),
            zcatalog=sdss_z,
            model=(sdss.spectral_axis.value, sdss.meta['model'].to(u.Unit('1e-17 erg/(cm**2 s Angstrom)'))),
            notebook=True, title=os.path.basename('SDSS Plate %d' % plate), model_from_zcat=False, with_coaddcam=False, mask_type='PRIMTARGET', with_thumb_tab=False, with_vi_widgets=False)

### Visualization Interface Details

* The primary visualization area consists of three plots:
  - The flux versus wavelength in the large area on the left.
    * Pan and zoom functions can be adjusted by clicking on the icons on the upper right.
    * Display of values can be toggled by clicking on the names in the legend in the upper right.  For example, to hide the noise spectrum, click on 'noise' in the legend.
  - The zoom display on the lower right.  This plot is automatically updated when the mouse is hovered or moved over the large spectrum display on the left.
  - The targeting image on the upper right.  This shows the image corresponding to the object.  The images are provided by the [Legacy Survey](https://www.legacysurvey.org/), and are actually deeper than SDSS images.  Click on the image to take you to the Legacy Survey Viewer.
* Target information table: this table displays information related to the *photometric object* associated with the spectrum.
  - `TARGETID`: For SDSS data, this tuple of numbers corresponds to the photometric imaging run, photometric processing version, camera column ("camcol"), imaging field (within the run), and object ID within the field.
  - Target class: This contains information about why the photometric object was selected for spectroscopic observation.  For SDSS data this is currently a dummy value, but will change in future versions of Prospect.
  - `mag_u`, ...: These are the photometric magnitudes in SDSS *u*, *g*, *r*, *i*, *z* filters.
* Reset X-Y range button: reset the display to the default pan and zoom values.
* OII-zoom button / Undo: pan and zoom the display to the position of the \[OII\] emission line.  This is not necessarily relevant for SDSS data.  "Undo" restores the previous pan/zoom values.
* Spectrum slider (< / >): Use these buttons or the slider to page through the loaded spectra.
* Pipeline fit (table): Displayes the classification and redshift obtained by the SDSS processing pipeline.
  - `SPECTYPE`: The type of object, typically 'GALAXY', 'QSO', 'STAR'.
  - `SUBTYPE`: A more specific classification of `SPECTYPE`, for example a 'STAR' may be 'K5' or a 'GALAXY' may be 'STARFORMING'.
  - `Z`: The redshift.
  - `ZERR`: The uncertainty on `Z`.
  - `ZWARN`: If the processing pipeline produced errors, this value will be non-zero.
  - $\Delta\chi^2$: the goodness of fit of the fitted model relative to the second-best model.
* Redshift box (orange): Use this box to adjust the redshift of the *model* on the display.
  - Rough and fine tuning adjust the displayed redshift in units of hundredths and ten-thousandths, respectively.
  - Reset to z_pipe: restore the displayed redshift to the redshift value obtained from the SDSS pipeline.
  - Copy z to VI: this button is disabled.
* Gaussian Sigma: SDSS spectra are inherently faint and noisy.  Smoothing will reduce the "jaggedness" caused by noise.  This is especially useful when comparing to the fitted model or investigating spectra lines.
* Emission lines / Absorption lines: Toggle the display of known spectral lines.  If "Show only major lines" is checked, only the spectral lines most likely to be visible are displayed.
* Obs / Rest: Display flux as a function of observed wavelength or rest wavelength, respectively.

## SDSS Spectra and Redshifts from Files

For reference, and to compare timing, below is how one would fetch spectra and redshift metadata from file-based storage.  First, obtain the path to the files in the Astro Data Lab file service.

In [7]:
spectro_redux = 'sdss_dr16://' + os.path.join('sdss', 'spectro', 'redux')
run2d = '26'
plate = 2955
mjd = '54562'
sdss_spectra = os.path.join(spectro_redux, run2d, str(plate), f'spPlate-{plate:d}-{mjd}.fits')
sdss_redshifts = os.path.join(spectro_redux, run2d, str(plate), f'spZbest-{plate:d}-{mjd}.fits')
print(sdss_spectra)
print(sdss_redshifts)

sdss_dr16://sdss/spectro/redux/26/2955/spPlate-2955-54562.fits
sdss_dr16://sdss/spectro/redux/26/2955/spZbest-2955-54562.fits


Next, read the spPlate file.  This automatically loads the [targeting metadata](#Target-Catalog).

In [8]:
%%time
sdss_file = read_spPlate(sc.get(sdss_spectra, mode='fileobj'), limit=50)
sdss_file

CPU times: user 347 ms, sys: 145 ms, total: 493 ms
Wall time: 1.82 s


<Spectrum1D(flux=<Quantity [[11.0880165 , 11.084556  , 11.081101  , ..., 11.787075  ,
            11.787052  , 11.787028  ],
           [ 4.371129  ,  4.36966   ,  4.3681927 , ...,  2.2523484 ,
             2.2523346 ,  2.2523208 ],
           [ 0.85558206,  0.8553027 ,  0.8550237 , ...,  8.030891  ,
             8.030855  ,  8.03082   ],
           ...,
           [12.411868  , 12.408562  , 12.405259  , ..., 25.452364  ,
            25.452513  , 25.45266   ],
           [-1.97795   , -1.9774021 , -1.976855  , ..., 15.674774  ,
            15.674834  , 15.674895  ],
           [ 0.6398605 ,  0.63967246,  0.63948476, ..., -0.11592854,
            -0.11592867, -0.1159288 ]] 1e-17 erg / (Angstrom cm2 s)>, spectral_axis=<SpectralAxis [3802.76948244, 3803.64520329, 3804.5211258 , ..., 9217.22098659,
   9219.34357452, 9221.46665124] Angstrom>, uncertainty=InverseVariance([[0., 0., 0., ..., 0., 0., 0.],
                 [0., 0., 0., ..., 0., 0., 0.],
                 [0., 0., 0., ..., 0., 0.,

Read the spZbest file, which contains the [redshift catalog](#Redshift-Catalog).

In [9]:
sdss_z_file, sdss_model_file = read_spZbest(sc.get(sdss_redshifts, mode='fileobj'), limit=50)

Launch the visualization.  Reading from files produces slightly different scaling on the flux values, but otherwise the options are the same as the example above.

In [ ]:
plotspectra(sdss_file,
            zcatalog=sdss_z_file,
            model=(sdss_model_file.spectral_axis.value, sdss_model_file.flux.value,),
            notebook=True, title=os.path.basename('SDSS Plate %d' % plate), model_from_zcat=False, with_coaddcam=False, mask_type='PRIMTARGET', with_thumb_tab=False, with_vi_widgets=False)

## References & Resources

* [Getting Started with Spectral Data](https://github.com/noaodatalab/specserver/blob/master/doc/03_GettingStartedWithSpectra.ipynb)

In [10]:
end_time = time.time()
print('Notebook run time: %.3g sec' % (end_time - start_time))

Notebook run time: 85.1 sec
